# Model Validation: The Bias-Variance Tradeoff and the Train-Test Split

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

## Agenda

SWBAT:

- explain the bias-variance tradeoff and the correlative notions of underfit and overfit models;
- describe a train-test split and explain its purpose in the context of predictive statistics / machine learning;
- explain the algorithm of cross-validation.

## The Bias-Variance Tradeoff

**High-bias** algorithms tend to be less complex, with simple or rigid underlying structure.

+ They train models that are consistent, but inaccurate on average.
+ These include linear or parametric algorithms such as regression and naive Bayes.
+ The following sorts of difficulties could lead to high bias:
      - We did not include the correct predictors;
      - We did not take interactions into account;
      - We missed a non-linear (polynomial) relationship. 
      
High-bias models are generally **underfit**: The models have not picked up enough of the signal in the data. And so even though they may be consistent, they don't perform particularly well on the initial data, and so they will be consistently inaccurate.

On the other hand, **high-variance** algorithms tend to be more complex, with flexible underlying structure.

+ They train models that are accurate on average, but inconsistent.
+ These include non-linear or non-parametric algorithms such as decision trees and nearest-neighbor models.
+ The following sorts of difficulties could lead to high variance:
      - We included an unreasonably large number of predictors;
      - We created new features by squaring and cubing each feature.

High variance models are **overfit**: The models have picked up on the noise as well as the signal in the data. And so even though they may perform well on the initial data, they will be inconsistently accurate on new data.

While we build our models, we have to keep this relationship in mind.  If we build complex models, we risk overfitting our models.  Their predictions will vary greatly when introduced to new data.  If our models are too simple, the predictions as a whole will be inaccurate.   

The goal is to build a model with enough complexity to be accurate, but not too much complexity to be erratic.

![optimal](img/optimal_bias_variance.png)
http://scott.fortmann-roe.com/docs/BiasVariance.html

### Let's take a look at our familiar King County housing data. 

In [ ]:
np.random.seed(42)
df = pd.read_csv('data/king_county.csv', index_col='id')
df = df.iloc[:, :12]
df.head()

In [ ]:
df.shape

In [ ]:
np.random.seed(42)

# Let's generate random subsets of our data

# Date is not in the correct format so we are dropping it for now.

sample_point = df.drop('price', axis=1).sample(1)
point_preds = []

r_2 = []
simple_rmse = []

for i in range(100):
    
    df_sample = df.sample(5000, replace=True)
    y = df_sample.price
    X = df_sample.drop('price', axis=1)
    
    lr = LinearRegression()
    lr.fit(X, y)
    
    y_hat = lr.predict(X)
    simple_rmse.append(np.sqrt(mean_squared_error(y, y_hat)))
    r_2.append(lr.score(X, y))
    
    y_hat_point = lr.predict(sample_point)
    
    point_preds.append(y_hat_point)

In [ ]:
print(f'simple mean: {np.mean(simple_rmse)}')
print(f'simple variance: {np.var(point_preds)}')

In [ ]:
df = pd.read_csv('data/king_county.csv', index_col='id')

pf = PolynomialFeatures(2)

df_poly = pd.DataFrame(pf.fit_transform(df.drop('price', axis=1)))
df_poly.index = df.index
df_poly['price'] = df['price']

cols = list(df_poly)

# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('price')))

df_poly = df_poly.loc[:, cols]

df_poly.head(10)

In [ ]:
np.random.seed(42)

sample_point = df_poly.drop('price', axis=1).sample(1)

r_2 = []
point_preds_comp = []
complex_rmse = []
for i in range(100):
    
    df_sample = df_poly.sample(1000, replace=True)
    y = df_sample.price
    X = df_sample.drop('price', axis=1)
    
    lr = LinearRegression()
    lr.fit(X, y)
    y_hat = lr.predict(X)
    complex_rmse.append(np.sqrt(mean_squared_error(y, y_hat)))
    r_2.append(lr.score(X, y))
    
    y_hat_point = lr.predict(sample_point)
    
    point_preds_comp.append(y_hat_point)

In [ ]:
print(f'simple mean {np.mean(simple_rmse)}')
print(f'complex mean {np.mean(complex_rmse)}')

print(f'simple variance {np.var(point_preds)}')
print(f'complex variance {np.var(point_preds_comp)}')

![which_model](img/which_model_is_better_2.png)

## Train-Test Split

It is hard to know if your model is too simple or complex by just using it on training data.

We can hold out part of our training sample, and use it as a test sample and use it to monitor our prediction error.

This allows us to evaluate whether our model has the right balance of bias/variance. 

<img src='img/testtrainsplit.png' width =550 />

* **training set** —a subset to train a model.
* **test set**—a subset to test the trained model.


In [ ]:
df = pd.read_csv('data/king_county.csv', index_col='id')

y = df.price
X = df[['bedrooms', 'sqft_living']]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=None,
                                                   random_state=42)

print(X_train.shape)
print(X_test.shape)

print(X_train.shape[0] == y_train.shape[0])
print(X_test.shape[0] == y_test.shape[0])

### How do we know if our model is overfitting or underfitting?

If our model is not performing well on the training  data, we are probably underfitting it.  


To know if our  model is overfitting the data, we need  to test our model on unseen data. 
We then measure our performance on the unseen data. 

If the model performs significantly worse on the  unseen data, it is probably  overfitting the data.

<img src='https://developers.google.com/machine-learning/crash-course/images/WorkflowWithTestSet.svg' width=500/>

### Examples

Consider the following scenarios and describe them according to bias and variance. There are four possibilities:

- a. The model has low bias and high variance.
- b. The model has high bias and low variance.
- c. The model has both low bias and low variance.
- d. The model has both high bias and high variance.

**Scenario 1**: The model has a low RMSE on training and a low RMSE on test.
<details>
    <summary> Anwer
    </summary>
    c. The model has both low bias and low variance.
    </details>

**Scenario 2**: The model has a high $R^2$ on the training set, but a low $R^2$ on the test.
<details>
    <summary> Anwer
    </summary>
    a. The model has low bias and high variance.
    </details>

**Scenario 3**: The model performs well on data it is fit on and well on data it has not seen.
<details>
    <summary> Anwer
    </summary>
    c. The model has both low bias and low variance.
    </details>
    
**Scenario 4**: The model has a low $R^2$ on training but high on the test set.
<details>
    <summary> Anwer
    </summary>
    d. The model has both high bias and high variance.
    </details>
    
**Scenario 5**: The model leaves out many of the meaningful predictors, but is consistent across samples.
<details>
    <summary> Anwer
    </summary>
    b. The model has high bias and low variance.
    </details>
    
**Scenario 6**: The model is highly sensitive to random noise in the training set.
<details>
    <summary> Anwer
    </summary>
    a. The model has low bias and high variance.
    </details>

### Should you ever fit on your test set?  


![no](https://media.giphy.com/media/d10dMmzqCYqQ0/giphy.gif)


**Never fit on test data.** If you are seeing surprisingly good results on your evaluation metrics, it might be a sign that you are accidentally training on the test set.

Let's go back to our KC housing data without the polynomial transformation.

In [ ]:
df = pd.read_csv('data/king_county.csv', index_col='id')

# Date  is not in the correct format so we are dropping it for now.
df.head()

Now, we create a train-test split via the `sklearn.model_selection` package.

In [ ]:
np.random.seed(42)

y = df.price
X = df[['bedrooms', 'sqft_living']]

# Here is the convention for a traditional train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
# Instanstiate your linear regression object
lr = LinearRegression()

In [ ]:
# fit the model on the training set
lr.fit(X_train, y_train)

In [ ]:
# Check the R^2 of the training data
lr.score(X_train, y_train)

In [ ]:
lr.coef_

A .506 R-squared reflects a model that explains about half of the total variance in the data. 

### Now check performance on test data

Next, we test how well the model performs on the unseen test data. Remember, we do not fit the model again. The model has calculated the optimal parameters learning from the training set.  


In [ ]:
lr.score(X_test, y_test)

### Knowledge check
How would you describe the bias of the model based on the above training $R^2$?

The difference between the train and test scores is low.

What does that indicate about variance?

### Same procedure with a polynomial model

In [ ]:
df = pd.read_csv('data/king_county.csv', index_col='id')
df.head()

In [ ]:
poly_2 = PolynomialFeatures(4)

X_poly = pd.DataFrame(
            poly_2.fit_transform(df.drop('price', axis=1))
                      )

y = df.price
X_poly.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y,
                                                    test_size=0.2,
                                                    random_state=42)
lr_poly = LinearRegression()

# Always fit on the training set
lr_poly.fit(X_train, y_train)

lr_poly.score(X_train, y_train)

In [ ]:
lr_poly.score(X_test, y_test)

### Exercise

[This post about scaling and data leakage](https://datascience.stackexchange.com/questions/38395/standardscaler-before-and-after-splitting-data) explains that if you are going to scale your data, you should only train your scaler on the training data to prevent data leakage.  

Perform the same train-test split as shown above for the simple model, but now scale your data appropriately.  

The $R^2$ for both train and test should be the same as before.

In [ ]:
np.random.seed(42)

y = df.price
X = df[['bedrooms', 'sqft_living']]

# Train test split with random_state=42 and test_size=0.2

# Scale appropriately

# fit and score the model 


## Kfolds: Even More Rigorous Validation  

For a more rigorous cross-validation, we turn to K-folds

![kfolds](img/k_folds.png)

[image via sklearn](https://scikit-learn.org/stable/modules/cross_validation.html)

In this process, we split the dataset into train and test as usual, then we perform a shuffling train-test split on the training set.  

KFolds holds out one fraction of the dataset, trains on the larger fraction, then calculates a test score on the held out set. It repeats this process until each group has served as the test set.

We tune our parameters on the training set using k-many folds, then validate on the test data. This allows us to build our model and check to see if it is overfit without touching the test data set. This protects our model from bias.

In [ ]:
X = df.drop('price', axis=1)
y = df.price

In [ ]:
kf = KFold(n_splits=5)

train_r2 = []
test_r2 = []
for train_ind, test_ind in kf.split(X, y):
    
    X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
    X_test, y_test = X.iloc[test_ind], y.iloc[test_ind]
    
    lr.fit(X_train, y_train)
    train_r2.append(lr.score(X_train, y_train))
    test_r2.append(lr.score(X_test, y_test))

In [ ]:
# Mean train r_2
np.mean(train_r2)

In [ ]:
# Mean test r_2
np.mean(test_r2)

In [ ]:
# Test out our polynomial model
poly_2 = PolynomialFeatures(2)

df_poly = pd.DataFrame(
            poly_2.fit_transform(df.drop('price', axis=1))
                      )

X = df_poly
y = df.price

In [ ]:
kf = KFold(n_splits=5)

train_r2 = []
test_r2 = []
for train_ind, test_ind in kf.split(X, y):
    
    X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
    X_test, y_test = X.iloc[test_ind], y.iloc[test_ind]
    
    lr.fit(X_train, y_train)
    train_r2.append(lr.score(X_train, y_train))
    test_r2.append(lr.score(X_test, y_test))

In [ ]:
# Mean train r_2
np.mean(train_r2)

In [ ]:
# Mean test r_2
np.mean(test_r2)

Once we have an acceptable model, we train our model on the entire training set, and score on the test to validate.

